In [39]:
# install devtools from https://github.com/hadley/devtools
#devtools::install_github("vz-risk/verisr")
library(tidyverse)
library(ggplot2)
library("scales")
library(dplyr)
library(verisr)


In [40]:
devtools::install_github("vz-risk/verisr")

Skipping install of 'verisr' from a github remote, the SHA1 (18cd3e86) has not changed since last install.
  Use `force = TRUE` to force installation



In [41]:
data('industry2', package='verisr')
vcdb.dir <- "VCDB/data/json/validated"
veris <- json2veris(vcdb.dir, schema="VCDB/vcdb-merged.json")

Warning message in json2veris(vcdb.dir, schema = "VCDB/vcdb-merged.json"):
"Column[s]: 
  "action.physical.vector.Privileged access", 
Not found in schema, source file:VCDB/data/json/validated/003c2a32-a95b-499b-9e9c-ea268df7bf1a.json"
Warning message in json2veris(vcdb.dir, schema = "VCDB/vcdb-merged.json"):
"Column[s]: 
  "plus.attack_difficulty_initial.Not Applicable", 
Not found in schema, source file:VCDB/data/json/validated/004a9681-f5bb-4c26-9972-97969fcdf2b0.jsonColumn[s]: 
  "plus.attack_difficulty_legacy.Not Applicable", 
Not found in schema, source file:VCDB/data/json/validated/004a9681-f5bb-4c26-9972-97969fcdf2b0.jsonColumn[s]: 
  "plus.attack_difficulty_subsequent.Not Applicable", 
Not found in schema, source file:VCDB/data/json/validated/004a9681-f5bb-4c26-9972-97969fcdf2b0.json"
Warning message in json2veris(vcdb.dir, schema = "VCDB/vcdb-merged.json"):
"Column[s]: 
  "plus.analysis_status.Needs review", 
Not found in schema, source file:VCDB/data/json/validated/00fd327e-

[1] "veris dimensions"
[1] 9009 2548
        action.physical.vector.Privileged access 
                                            2499 
   plus.attack_difficulty_initial.Not Applicable 
                                            2500 
    plus.attack_difficulty_legacy.Not Applicable 
                                            2501 
plus.attack_difficulty_subsequent.Not Applicable 
                                            2502 
               plus.analysis_status.Needs review 
                                            2503 
    action.hacking.variety.Use of backdoor or C2 
                                            2504 
            action.hacking.vector.Backdoor or C2 
                                            2505 
                    action.social.vector.Website 
                                            2506 
             action.hacking.variety.Footprinting 
                                            2507 
named integer(0)


In [46]:
summary(veris)

9009 incidents in this object.


Warning message:
"`getenumCI2019()` was deprecated in verisr version 2.3.1.0005.
Please use `getenumCI2020()` instead."


    actor              action             asset            attribute        
 Length:13205       Length:13205       Length:13205       Length:13205      
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  

In [156]:
df1<-veris%>%
    filter(plus.dbir_year %in% c(2017:2021))%>% # filter to years 2017 - 2021
    filter(attribute.confidentiality.data_disclosure.Yes)%>%# filter to look at confirmed breaches only
    getenumCI("victim.employee_count")%>%
    filter(!enum %in% c("Small","Large","Unknown"))%>%
    mutate(n=sum(x))%>%
    mutate(freq=x/n)

# Combine 1-10 with 11-100 to get data for less than 100 employees
df2<-df1%>%
    filter(enum %in% c("1 to 10","11 to 100"))%>%
    mutate(enum="Less than 100")%>%
    mutate(freq=round(freq,2))%>%
    mutate(x=sum(x))%>%
    mutate(freq=sum(freq))%>%
    distinct(x, .keep_all=TRUE)

df3<-df1%>%filter(enum %in% c("10001 to 25000","25001 to 50000", "50001 to 100000"))%>%
    mutate(freq=sum(freq))%>%
    mutate(enum="10,001 to 100,000")%>%
    mutate(x=sum(x))%>%
    distinct(freq, .keep_all=TRUE)

df4<-df1%>%filter(!enum%in% c("1 to 10","11 to 100","10001 to 25000","25001 to 50000", "50001 to 100000"))
df<-rbind(df2,df3,df4)#%>%mutate(freq=percent(freq,accuracy=1))
df<-df%>%mutate(freq=percent(freq,accuracy=1))
print('Company data breaches broken down by company
print(df)

               enum   x    n freq
1     Less than 100 313 1279  25%
2 10,001 to 100,000 215 1279  17%
3       101 to 1000 356 1279  28%
4     1001 to 10000 326 1279  25%
5       Over 100000  69 1279   5%
